In [19]:
import torch
torch.manual_seed(0)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

import random
random.seed(0)

import numpy as np
np.random.seed(0)

import sys
import os
import re

# load packages
import time
import random
import yaml
from munch import Munch
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torchaudio
import librosa
from nltk.tokenize import word_tokenize, sent_tokenize

import spacy
from spacy.lang.en import English

from models import *
from utils import *
from text_utils import TextCleaner
textclenaer = TextCleaner()

%matplotlib inline

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def length_to_mask(lengths):
    mask = torch.arange(lengths.max()).unsqueeze(0).expand(lengths.shape[0], -1).type_as(lengths)
    mask = torch.gt(mask+1, lengths.unsqueeze(1))
    return mask

# load phonemizer
import phonemizer
global_phonemizer = phonemizer.backend.EspeakBackend(language='en-us', preserve_punctuation=True,  with_stress=True)

config = yaml.safe_load(open("Models/LJSpeech/config.yml"))

# load pretrained ASR model
ASR_config = config.get('ASR_config', False)
ASR_path = config.get('ASR_path', False)
text_aligner = load_ASR_models(ASR_path, ASR_config)

# load pretrained F0 model
F0_path = config.get('F0_path', False)
pitch_extractor = load_F0_models(F0_path)

# load BERT model
from Utils.PLBERT.util import load_plbert
BERT_path = config.get('PLBERT_dir', False)
plbert = load_plbert(BERT_path)

177


In [14]:
passage = '''He began a sentence: “I am–” but when he was taken by surprise, every sentence became an adventure in the woods; as soon as he could no longer see the light of the clearing from which he’d entered, he would realize that the crumbs he’d dropped for bearings had been eaten by birds, silent deft darting things which he couldn’t quite see in the darkness but which were so numerous and swarming in their hunger that it seemed as if they were the darkness, as if the darkness weren’t uniform, weren’t an absence of light but a teeming corpuscular thing, and indeed when as a studious teenager he’d encountered the word “crepuscular” in McKay’s Treasury of English Verse, the corpuscles of biology had bled into his understanding of the word, so that for his entire adult life he’d seen in twilight a corpuscularity, as of the graininess of the high-speed film necessary for photography under conditions of low ambient light, as of a kind of sinister decay; and hence the panic of a man betrayed deep in the woods whose darkness was the darkness of starlings blotting out the sunset or black ants storming a dead opossum, a darkness that didn’t just exit but actively consumed the bearings that he’d sensibly established for himself, lest he be lost; but in the instant of realizing he was lost, time became marvelously slow and he discovered hitherto unguessed eternities in the space between one word and the next, or rather he became trapped in that space between one word and the next, or rather he became trapped in that space between words and could only stand and watch as time sped on without him, the thoughtless boyish part of him crashing on out of sight blindly through the woods while he, trapped, the grownup Al, watched in oddly impersonal suspense to see if the panic-stricken little boy might, despite no longer knowing where he was or at what point he’d entered the woods of this sentence, still manage to blunder into the clearing where Enid was waiting for him, unaware of any woods–“packing my suitcase,” he heard himself say. '''
passage += '''Of all the problems which have been submitted to my friend, Mr.
Sherlock Holmes, for solution during the years of our intimacy, there
were only two which I was the means of introducing to his notice—that
of Mr. Hatherley’s thumb, and that of Colonel Warburton’s madness. Of
these the latter may have afforded a finer field for an acute and
original observer, but the other was so strange in its inception and so
dramatic in its details that it may be the more worthy of being placed
upon record, even if it gave my friend fewer openings for those
deductive methods of reasoning by which he achieved such remarkable
results. The story has, I believe, been told more than once in the
newspapers, but, like all such narratives, its effect is much less
striking when set forth _en bloc_ in a single half-column of print than
when the facts slowly evolve before your own eyes, and the mystery
clears gradually away as each new discovery furnishes a step which
leads on to the complete truth. At the time the circumstances made a
deep impression upon me, and the lapse of two years has hardly served
to weaken the effect.'''


In [50]:
print(passage)

He began a sentence: “I am–” but when he was taken by surprise, every sentence became an adventure in the woods; as soon as he could no longer see the light of the clearing from which he’d entered, he would realize that the crumbs he’d dropped for bearings had been eaten by birds, silent deft darting things which he couldn’t quite see in the darkness but which were so numerous and swarming in their hunger that it seemed as if they were the darkness, as if the darkness weren’t uniform, weren’t an absence of light but a teeming corpuscular thing, and indeed when as a studious teenager he’d encountered the word “crepuscular” in McKay’s Treasury of English Verse, the corpuscles of biology had bled into his understanding of the word, so that for his entire adult life he’d seen in twilight a corpuscularity, as of the graininess of the high-speed film necessary for photography under conditions of low ambient light, as of a kind of sinister decay; and hence the panic of a man betrayed deep in 

In [17]:
def phoneme_tokenize(s):
    # text = passage.split('.')[0].strip()
    text = s.strip()

    # text = text.replace('"', '')
    ps = global_phonemizer.phonemize([text])
    ps = word_tokenize(ps[0])
    ps = ' '.join(ps)
    tokens = textclenaer(ps)
    tokens.insert(0, 0)
    return tokens



In [51]:
# Function to split text into tokens, if the text generates more than 512 tokens (downstream network cannot process more than 512 tokens at once)
# We first split the text into pieces by punctuation. 
# If the pieces are still too long, we'll split on spaces into equal size pieces.
# If it's still too long, we truncate the tokens, to avoid the network failing.
def subtokenize(s):
    txt = re.split(r"[,;]", s)
    if len(txt)>1:
        token_list=[]
        for t in txt:
            token_list += subtokenize(t)
        return token_list
    else:
        token = phoneme_tokenize(txt[0])
        if len(token) < 511:
            print('\t\t',txt[0],'\n\n')
            return [token]
        else:
            t = re.split(r"[ ]", txt[0])
            length = len(t) # number of words
            if length == 1:
                token = phoneme_tokenize(t[0])
                if len(token) > 511:
                    print("still too long {token}")
                    return [token[:511]]
                else :
                    return [token]
            else:
                return subtokenize(" ".join(t[0:length//2])) + subtokenize(" ".join(t[length//2:]))



def tokenize_text(passage):
    for s in sent_tokenize(passage):
        s=s.replace('\n','').replace('\r','')
        tokens = phoneme_tokenize(s)
        if len(tokens) < 511:
            print(s,'\n\n')
            tokens = [tokens]
        else:
            tokens=subtokenize(s)

        for t in tokens:
            print(len(t))



		 He began a sentence: “I am–” but when he was taken by surprise 


		  every sentence became an adventure in the woods 


		  as soon as he could no longer see the light of the clearing from which he’d entered 


		  he would realize that the crumbs he’d dropped for bearings had been eaten by birds 


		  silent deft darting things which he couldn’t quite see in the darkness but which were so numerous and swarming in their hunger that it seemed as if they were the darkness 


		  as if the darkness weren’t uniform 


		  weren’t an absence of light but a teeming corpuscular thing 


		  and indeed when as a studious teenager he’d encountered the word “crepuscular” in McKay’s Treasury of English Verse 


		  the corpuscles of biology had bled into his understanding of the word 


		  so that for his entire adult life he’d seen in twilight a corpuscularity 


		  as of the graininess of the high-speed film necessary for photography under conditions of low ambient light 


		  as of a k

In [42]:
a=re.split(r"[ ]", 'blah, blah')
len(a)
print(a)

['blah,', 'blah']


In [8]:
len(tokens)

2170

In [9]:
with open("testtext.txt", 'r') as sourcefile:
    text = sourcefile.read()
    text = text.replace('\n', ' ').replace('\r', ' ')
    

In [22]:
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('sentencizer') 
doc = nlp(text)
sentences = [sent.text.strip() for sent in doc.sents]
print(sentences)

['', 'I took it up and glanced at it.', '“Mr. Victor Hatherley, hydraulic engineer, 16A, Victoria Street (3rd floor).”', 'That was the name, style, and abode of my morning visitor.', '“I regret that I have kept you waiting,” said I, sitting down in my library-chair.', '“You are fresh from a night journey, I understand, which is in itself a monotonous occupation.”', '“I saw the teeming sea; I saw daybreak and nightfall; I saw the multitudes of America; I saw a silvery cobweb in the center of a black pyramid; I saw a splintered labyrinth (it was London); I saw, close up, unending eyes watching themselves in me as in a mirror; I saw all the mirrors on earth and none of them reflected me; I saw in a backyard of Soler Street the same tiles that thirty years before I’d seen in the entrance of a house in Fray Bentos; I saw bunches of grapes, snow, tobacco, lodes of metal, steam; I saw convex equatorial deserts and each one of their grains of sand; I saw a woman in Inverness whom I shall never

In [29]:
t=list(nlp.tokenizer(sentences[2]))
print(t)

[“, Mr., Victor, Hatherley, ,, hydraulic, engineer, ,, 16A, ,, Victoria, Street, (, 3rd, floor, ), ., ”]


In [28]:
for w in t:
    print(w)

“
Mr.
Victor
Hatherley
,
hydraulic
engineer
,
16A
,
Victoria
Street
(
3rd
floor
)
.
”
